<a href="https://colab.research.google.com/github/marcekovam/Data_science_practicum_2025/blob/main/Lesson2_FashionMNIST_Dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's build a simple neural network to classify images from the FashionMNIST dataset.

**1. Import Libraries**

In [ ]:
import torch
import torch.nn as nn # pytorch.nn won against tensorflow
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

*Checking for GPU Availability*

This code checks if a CUDA-enabled GPU is available and sets the `device` accordingly. If no GPU is available, it defaults to the CPU.

In [ ]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


**2. Data Preparation**

In [ ]:
# Define a transform to convert images to tensors
transform = transforms.ToTensor()

# Download and load the training data
# Data set = List of files
# Data loader = loading  data to memory (separately because of the size of big data)
# Importatnt number ... batch_size ... number of pictures to be loaded at the same time
# shuffle = True ... randomizes the batches
train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True)

# Download and load the test data
test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)

100%|██████████| 26.4M/26.4M [00:00<00:00, 113MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 3.73MB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 56.4MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.7MB/s]


**3. Neural Network Model**

In [ ]:
# Modele is a neural network (NN) or a part of it
# NN needs:
## Initial values of parameters
## Data inputs and outputs (forward)
## Derivation of outcome against the inputs (backward)

class DenseNet(nn.Module):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)  # fc =  fully connected
        self.fc2 = nn.Linear(128, 10)
        # 28*28 is the size of each picture, number of input nodes
        # 1st layer has 128 neurons
        # output layer has 10 categories

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))  # relu creates non-linearity in NN
        logits = self.fc2(x)  # computes logit from 3rd layer
        return logits

model = DenseNet().to(device)
model

DenseNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

**4. Loss & Optimizer**

In [ ]:
criterion = nn.CrossEntropyLoss()  # choose loss function
optimizer = optim.SGD(model.parameters(), lr=0.002)
# STG = stochastic gradient descent
# lr = learning rate ... usually we have to make a guess, usually small lr is appropriate

**5. Training loop**

In [ ]:
# Epoch ... how many times we use the whole dataset (each picture in it)

for epoch in range(5):  # Train for 5 epochs
    running_loss = 0.0
    for images, labels in train_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()  # delete the gradient

        # Forward pass
        outputs = model(images)  # pass images to model
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()  # compute the gradient
        optimizer.step()  # change the parameters to "better" values

        running_loss += loss.item()  # loss should be descreasing! if there is no underlaying problem

    print(f'Epoch [{epoch+1}/5], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/5], Loss: 1.2586
Epoch [2/5], Loss: 1.1570
Epoch [3/5], Loss: 1.0800
Epoch [4/5], Loss: 1.0200
Epoch [5/5], Loss: 0.9719


**6. Evaluation on the test set**

In [ ]:
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for evaluation
    for images, labels in test_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 67.19%
